In [1]:
import numpy as np
from basic.mnist import Mnist

In [2]:
# load data from Mnist Dataset
dataloader = Mnist()
train_data, train_label = dataloader.get_data(train=True)

In [3]:
np.array(train_data).shape

(60000, 784)

### Loss function of Logistic Regression:
$$
\scalebox{1.5}{
\begin{align*}
J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log\left(h_\theta(x^{(i)})\right) + (1 - y^{(i)}) \log\left(1 - h_\theta(x^{(i)})\right) \right]
\end{align*}
}
$$

### Gradient Calculation:
$$\scalebox{1.5} {
\begin{align*}
g_i = \frac{\partial J(\theta)} {\partial w_i}=(p(x_i) - y_i)x_i
\end{align*}
}
$$

### how to calculate p(x_i)
$$\scalebox{1.5} {
\begin{align*}
p(x_i) = \frac{1}{1 + e^{-(w^Tx + b)}}
\end{align*}
}
$$

In [4]:
def logistic_regression(epochs=10, step = 0.001):
    """
    :param epochs: 迭代次数
    :param step: 步长
    :return:
    """
    for i in range(len(train_data)):
        train_data[i].append(1)

    train_data_arr = np.array(train_data)

    # 初始化w,
    w = np.zeros(train_data_arr.shape[1])

    # 迭代轮数
    for epoch in range(epochs):
        # 遍历数据进行调参
        if epoch % 10 == 0:
            print(epoch)
        for idx in range(train_data_arr.shape[0]):
            x= train_data_arr[idx]
            y = train_label[idx]
            wx = np.dot(w, x)
            # 梯度下降
            px = 1 / (1 + np.exp(-wx))
            gradient = (px - y) * px
            w += step * gradient

    return w

In [5]:
def pred(w, x):
    wx = np.dot(w, x)
    p = np.exp(wx) / (1 + np.exp(wx))
    if p > 0.5:
        return 1
    else:
        return 0

In [6]:
def test(w):
    test_data, test_label = dataloader.get_data(False)

    err_cnt = 0

    for i in range(len(test_data)):
        test_data[i].append(1)

    for idx in range(len(test_data)):
        if test_label[idx] != pred(w, test_data[idx]):
            err_cnt += 1

    return round(1 - err_cnt / len(test_data), 2)

In [7]:
w = logistic_regression()

0


In [8]:
w.shape

(785,)